In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0

In [1]:
import pandas as pd
import numpy as np
import re #регулярные выражения
from nltk.stem.snowball import RussianStemmer # стемминг
from nltk.stem import WordNetLemmatizer # лемматизация
from nltk.tokenize import word_tokenize # токенизация текста (разбиение на слова/токены)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.corpus import stopwords # стопслова для удаления из текста
from sklearn.feature_extraction.text import TfidfVectorizer # векторизация текста
from sklearn.model_selection import train_test_split # разделение выборки
import torch
from torch import nn
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset # для создания датасета
from sklearn.metrics import f1_score
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Загрузка данных из датасета

In [2]:
df = pd.read_csv("insurance_reviews.csv")
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5,22.09.2024
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4,22.09.2024
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1,22.09.2024
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5,22.09.2024
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1,22.09.2024
...,...,...,...,...,...
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,Позвонили с Тинькофф Страхования. У меня оформ...,5,20.01.2024
9557,Мой выбор - Ренессанс,Ренессанс,Здравствуйте! Не всегда принято оставлять поло...,5,20.01.2024
9558,"Защита в пути, КАСКО",Сбербанк страхование,26.12.23 приобрёл автомобиль в кредит в Драйв ...,1,20.01.2024
9559,Нет возврата по страховке,Хоум Кредит Страхование,"Помогите пожалуйста , взял кредит и сразу отка...",1,20.01.2024


### Обработка данных

In [3]:
df.shape

(9561, 5)

In [4]:
df.dtypes

,0
header,object
bank,object
review,object
rate,int64
date,object


In [5]:
df.describe()

,rate
count,9561.000000
mean,3.579856
std,1.865433
min,1.000000
25%,1.000000
50%,5.000000
75%,5.000000
max,5.000000


In [6]:
df.nunique()

,0
header,7105
bank,65
review,9551
rate,5
date,485


In [7]:
df["review"].duplicated().sum()

10

In [8]:
df.isna().sum()

,0
header,0
bank,0
review,0
rate,0
date,0


В результате изучения датасета и его наполнения выяснилось:

* размерность датасета - `(9561, 5)`;
* оценка указана в числовом формате `int64`, остальные столбцы - `object`;
* средняя оценка по отзывам - `3,58`;
* из 9561 отзывов уникальными являются 9551, 10 отзывов - дубликаты;
* нулевые значения отсутствуют;
* количество банков/организаций в датасете - `65`.

Следовательно, дубликаты необходимо удалить. Далее будет обработка и очистка данных.

In [9]:
# удаляем дубликаты отзывов
df.drop_duplicates(subset="review", inplace=True)
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5,22.09.2024
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4,22.09.2024
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1,22.09.2024
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5,22.09.2024
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1,22.09.2024
...,...,...,...,...,...
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,Позвонили с Тинькофф Страхования. У меня оформ...,5,20.01.2024
9557,Мой выбор - Ренессанс,Ренессанс,Здравствуйте! Не всегда принято оставлять поло...,5,20.01.2024
9558,"Защита в пути, КАСКО",Сбербанк страхование,26.12.23 приобрёл автомобиль в кредит в Драйв ...,1,20.01.2024
9559,Нет возврата по страховке,Хоум Кредит Страхование,"Помогите пожалуйста , взял кредит и сразу отка...",1,20.01.2024


In [10]:
# изменение формата столбца даты с обходом и заменой строк, в которых дата указана некорректно
# (некорректные данные будут заменены на значение NaT)
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y", errors='coerce')
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1,2024-09-22
...,...,...,...,...,...
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,Позвонили с Тинькофф Страхования. У меня оформ...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,Здравствуйте! Не всегда принято оставлять поло...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,26.12.23 приобрёл автомобиль в кредит в Драйв ...,1,2024-01-20
9559,Нет возврата по страховке,Хоум Кредит Страхование,"Помогите пожалуйста , взял кредит и сразу отка...",1,2024-01-20


При изменении формата данных обнаружилось, что не во всех строках указана корректная дата. Посмотрим, сколько строк содержат ошибочные данные и удалим.

In [11]:
# количество строк с некорректной датой
df["date"].isna().sum()

1395

In [12]:
df.dropna(axis=0, subset="date", inplace=True)
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1,2024-09-22
...,...,...,...,...,...
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,Позвонили с Тинькофф Страхования. У меня оформ...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,Здравствуйте! Не всегда принято оставлять поло...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,26.12.23 приобрёл автомобиль в кредит в Драйв ...,1,2024-01-20
9559,Нет возврата по страховке,Хоум Кредит Страхование,"Помогите пожалуйста , взял кредит и сразу отка...",1,2024-01-20


Проверим количество отзывов для каждой организации и удалим те, у которых количество отзывов меньше 100.

In [13]:
df_bank = df.groupby("bank").review.count().sort_values()
df_bank

,review
bank,
Ак Барс Страхование,1
Росбанк Страхование,1
РБ Страхование Жизни,1
Ю-Лайф,1
НКО ПОВС «Страховой дом «Платинум»,1
...,...
Ренессанс Жизнь,383
АльфаСтрахование-Жизнь,399
Сбербанк страхование,939


In [14]:
organizations_less_100_reviews = df_bank.loc[df_bank.index.isin(df_bank.where(df_bank < 100).dropna().index)].index
organizations_less_100_reviews

Index(['Ак Барс Страхование', 'Росбанк Страхование', 'РБ Страхование Жизни',
       'Ю-Лайф', 'НКО ПОВС «Страховой дом «Платинум»',
       'Страховая фирма «Адонис»', 'Зетта Страхование жизни',
       'Зетта Страхование (бывшая СК «Альянс»)', 'Дефанс Страхование',
       'ПСБ Страхование', 'Пари',
       'Медицинская акционерная страховая компания (МАКС-М)',
       'Капитал МС (РГС-Медицина)', 'Двадцать первый век',
       'Русский Стандарт Страхование', 'АльфаСтрахование-ОМС',
       'ДЕЛО ЖИЗНИ (НСГ Страхование жизни)', 'Инсайт',
       'Русское страховое общество «Евроинс»', 'Бестиншур',
       'Инлайф страхование жизни', 'РЕСО-Мед', 'ППФ Страхование жизни',
       'Страховая бизнес группа', 'Газпром Страхование', 'Ингосстрах-Жизнь',
       'Объединенная страховая компания', 'Спасские ворота',
       'РСХБ-Страхование жизни', 'Энергогарант', 'БКС Страхование жизни',
       'РСХБ-Страхование', 'СОГАЗ-Мед', 'ВСК-Линия жизни',
       'Хоум Кредит Страхование', 'СК ГАЙДЕ', 'Капитал-Поли

In [15]:
df = df[df.bank.isin(list(organizations_less_100_reviews)) == False]
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1,2024-09-22
...,...,...,...,...,...
9554,Тинькофф страхование,Т-Страхование,"Все прошло быстро, деньги поступили на карту у...",5,2024-01-21
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,Позвонили с Тинькофф Страхования. У меня оформ...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,Здравствуйте! Не всегда принято оставлять поло...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,26.12.23 приобрёл автомобиль в кредит в Драйв ...,1,2024-01-20


In [16]:
# выведем среднее количество отзывов для организаций
mean_number_of_reviews = round(df.groupby("bank").review.count().mean(), 2)
mean_number_of_reviews

530.71

Выведем
* размерность датасета, очищенного от дубликатов, пустых значений, некорректной даты и организаций с малым количеством отзывов;
* количество уникальных значений для очищенного датасета;
* описательную характеристику нового датасета.


In [17]:
df.shape

(7430, 5)

In [18]:
df.describe()

,rate,date
count,7430.000000,7430
mean,3.734455,2024-06-11 00:18:36.339165696
min,1.000000,2024-01-20 00:00:00
25%,1.000000,2024-04-10 00:00:00
50%,5.000000,2024-06-22 00:00:00
75%,5.000000,2024-08-18 00:00:00
max,5.000000,2024-09-22 00:00:00
std,1.815010,NaN


In [19]:
mean_value = round(df.describe()["rate"]["mean"], 2)
mean_value

3.73

В результате обработки данных:
* размерность датасета стала `(7430, 5)` - удалили `2131` строки;
* средняя оценка по выборке стала `3.73` - выросла на `0,15`;
* были удалены дубликаты отзывов;
* были удалены строки, в котрых дата была указана неправильно;
* были удалены организации, у которых количество отзывов было меньше 100.

### Нормализация слов

Сначала напишем 2 функции, которые приведут все данные в нижний регистр, удалит стоп-слова и лишние символы, а так же проведет стемминг или лемматизацию.

In [20]:
df_stemming = df.copy()
df_lematization = df.copy()

In [21]:
def prepare_data_stem(text, window_size=2):
  '''Функция обработки текстовых полей для более удобной работы далее'''
  # преобразуем текст в нижний регистр
  text = text.lower()

  # удаляем все символы кроме букв и пробелов
  text = re.sub(r'[^а-я\s]', ' ', text)

  # удаляем стопслова
  tokens = word_tokenize(text) # делим строку на токены
  filtered_tokens = [word for word in tokens if not word in stopwords.words('russian')]

  # стемминг
  st = RussianStemmer()
  stemmed_tockens = [st.stem(word) for word in filtered_tokens]
  result = " ".join(stemmed_tockens)
  return result

In [22]:
df_stemming["review"] = df_stemming["review"].apply(prepare_data_stem)

In [23]:
df_stemming

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,оформ каск пожалел случ дтп менеджер анастас с...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,сам деш ва страховк совкомбанк позвон сам помо...,4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,здравств уважа сммщик т банк пришел знач сво л...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,добр вечер сво автомобил застрахова альфастрах...,5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,дорог человек ко случа связыва сбербанк люб ст...,1,2024-09-22
...,...,...,...,...,...
9554,Тинькофф страхование,Т-Страхование,прошл быстр деньг поступ карт втор ден подач д...,5,2024-01-21
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,позвон тинькофф страхован оформл ипотек днях д...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,здравств принят оставля положительн отзыв стра...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,приобр л автомобил кред драйв клик банк услов ...,1,2024-01-20


In [24]:
def prepare_data_lem(text, window_size=2):
  '''Функция обработки текстовых полей для более удобной работы далее'''
  # преобразуем текст в нижний регистр
  text = text.lower()

  # удаляем все символы кроме букв и пробелов
  text = re.sub(r'[^а-я\s]', ' ', text)

  # удаляем стопслова
  tokens = word_tokenize(text) # делим строку на токены
  filtered_tokens = [word for word in tokens if not word in stopwords.words('russian')]

  # лемматизация
  wnl = WordNetLemmatizer()
  lemmed_tockens = [wnl.lemmatize(word) for word in filtered_tokens]
  result = " ".join(lemmed_tockens)
  return result

In [25]:
df_lematization["review"] = df_lematization["review"].apply(prepare_data_lem)

In [26]:
df_lematization

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,оформили каско пожалели случилось дтп менеджер...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,самая деш вая страховка совкомбанк позвонили с...,4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,здравствуйте уважаемые сммщики т банка пришел ...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,добрый вечер свой автомобиль застраховал альфа...,5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,дорогие человеки коем случае связывайтесь сбер...,1,2024-09-22
...,...,...,...,...,...
9554,Тинькофф страхование,Т-Страхование,прошло быстро деньги поступили карту второй де...,5,2024-01-21
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,позвонили тинькофф страхования оформлена ипоте...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,здравствуйте принято оставлять положительные о...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,приобр л автомобиль кредит драйв клик банке ус...,1,2024-01-20


### Подготовка данных для моделей

#### Стемминг

In [27]:
# датасет с использованием стемминга
X_stemm = df_stemming["review"]
y_stemm = df_stemming["rate"].values - 1

In [28]:
# Векторизация
vectorizer_stemming = CountVectorizer()
X_stemming_vectorized = vectorizer_stemming.fit_transform(X_stemm).toarray()
X_stemming_tensor = torch.tensor(X_stemming_vectorized, dtype=torch.float32)
y_stemming_tensor = torch.tensor(y_stemm, dtype=torch.long)

In [29]:
dataset_stemming = TensorDataset(X_stemming_tensor, y_stemming_tensor)
dataloader_stemming = DataLoader(dataset_stemming, batch_size=32, shuffle=True)

#### Лемматизация

In [30]:
# датасет с использованием лемматизации
X_lemm = df_lematization["review"]
y_lemm = df_lematization["rate"].values - 1

In [31]:
# Векторизация для лемматизации
vectorizer_lemmatization = CountVectorizer()
X_lemmatization_vectorized = vectorizer_lemmatization.fit_transform(X_lemm).toarray()
X_lemmatization_tensor = torch.tensor(X_lemmatization_vectorized, dtype=torch.float32)
y_lemmatization_tensor = torch.tensor(y_lemm, dtype=torch.long)


In [32]:
dataset_lemmatization = TensorDataset(X_lemmatization_tensor, y_lemmatization_tensor)
dataloader_lemmatization = DataLoader(dataset_lemmatization, batch_size=32, shuffle=True)

### Модели

In [33]:
def evaluate_model(model, dataloader):
    model.eval()
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for reviews, targets in dataloader:
            outputs = model(reviews)
            _, predicted = torch.max(outputs.data, 1)

            all_targets.extend(targets.numpy())  # Собираем все целевые метки
            all_predictions.extend(predicted.numpy())  # Собираем все предсказания

    # Вычисляем F1-метрику
    f1 = f1_score(all_targets, all_predictions, average='weighted')  # Используем 'weighted' для многоклассовой классификации
    accuracy = sum(np.array(all_predictions) == np.array(all_targets)) / len(all_targets)  # Вычисляем точность

    return accuracy, f1

#### RNN

In [40]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.2):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x.unsqueeze(1))  # Добавляем размерность для последовательности
        out = self.fc(out[:, -1, :])  # Используем последний выход RNN
        return out

Обучение на данных, обработанных стеммингом

In [41]:
input_size = X_stemming_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_stemm))  # Количество классов

model_stemming_rnn = RNNModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_stemming_rnn.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_stemming:
        optimizer.zero_grad()
        outputs = model_stemming_rnn(reviews)
        loss_rnn_stem = criterion(outputs, targets)
        loss_rnn_stem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_rnn_stem.item():.4f}')

Epoch [1/10], Loss: 0.9756
Epoch [2/10], Loss: 0.0050
Epoch [3/10], Loss: 0.1314
Epoch [4/10], Loss: 0.0264
Epoch [5/10], Loss: 0.0049
Epoch [6/10], Loss: 0.0031
Epoch [7/10], Loss: 0.0026
Epoch [8/10], Loss: 0.0015
Epoch [9/10], Loss: 0.0044
Epoch [10/10], Loss: 0.0010


Обучение на данных, обработанных лемматизацией

In [42]:
input_size = X_lemmatization_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_lemm))  # Количество классов

model_lemm_rnn = RNNModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lemm_rnn.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_lemmatization:
        optimizer.zero_grad()
        outputs = model_lemm_rnn(reviews)
        loss_rnn_lem = criterion(outputs, targets)
        loss_rnn_lem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_rnn_lem.item():.4f}')

Epoch [1/10], Loss: 0.0880
Epoch [2/10], Loss: 0.0296
Epoch [3/10], Loss: 0.0341
Epoch [4/10], Loss: 0.0018
Epoch [5/10], Loss: 0.0020
Epoch [6/10], Loss: 0.0004
Epoch [7/10], Loss: 0.0017
Epoch [8/10], Loss: 0.0012
Epoch [9/10], Loss: 0.0006
Epoch [10/10], Loss: 0.0019


#### LSTM

In [37]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x.unsqueeze(1))  # Добавляем размерность для последовательности
        out = self.fc(out[:, -1, :])  # Используем последний выход RNN
        return out

Обучение на данных, обработанных стеммингом

In [38]:
input_size = X_stemming_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_stemm))  # Количество классов

model_stemming_lstm = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_stemming_lstm.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_stemming:
        optimizer.zero_grad()
        outputs = model_stemming_lstm(reviews)
        loss_lstm_stem = criterion(outputs, targets)
        loss_lstm_stem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_lstm_stem.item():.4f}')

Epoch [1/10], Loss: 0.0865
Epoch [2/10], Loss: 0.5012
Epoch [3/10], Loss: 0.0134
Epoch [4/10], Loss: 0.0138
Epoch [5/10], Loss: 0.0065
Epoch [6/10], Loss: 0.0227
Epoch [7/10], Loss: 0.0249
Epoch [8/10], Loss: 0.0051
Epoch [9/10], Loss: 0.0018
Epoch [10/10], Loss: 0.0021


Обучение на данных, обработанных лемматизацией

In [39]:
input_size = X_lemmatization_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_lemm))  # Количество классов

model_lemm_lstm = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lemm_lstm.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_lemmatization:
        optimizer.zero_grad()
        outputs = model_lemm_lstm(reviews)
        loss_lstm_lem = criterion(outputs, targets)
        loss_lstm_lem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_lstm_lem.item():.4f}')

Epoch [1/10], Loss: 0.1011
Epoch [2/10], Loss: 0.0533
Epoch [3/10], Loss: 0.0070
Epoch [4/10], Loss: 0.0040
Epoch [5/10], Loss: 0.0027
Epoch [6/10], Loss: 0.0012
Epoch [7/10], Loss: 0.0017
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0008
Epoch [10/10], Loss: 0.0008


#### GRU

In [34]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x.unsqueeze(1))  # Добавляем размерность для последовательности
        out = self.fc(out[:, -1, :])  # Используем последний выход GRU
        return out


Обучение на данных, обработанных стеммингом

In [35]:
input_size = X_stemming_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_stemm))  # Количество классов

model_stemming_gru = GRUModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_stemming_gru.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_stemming:
        optimizer.zero_grad()
        outputs = model_stemming_gru(reviews)
        loss_gru_stem = criterion(outputs, targets)
        loss_gru_stem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_gru_stem.item():.4f}')

Epoch [1/10], Loss: 0.0309
Epoch [2/10], Loss: 0.0701
Epoch [3/10], Loss: 0.0438
Epoch [4/10], Loss: 0.0111
Epoch [5/10], Loss: 0.0034
Epoch [6/10], Loss: 0.0019
Epoch [7/10], Loss: 0.0016
Epoch [8/10], Loss: 0.0022
Epoch [9/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0001


Обучение на данных, обработанных лемматизацией

In [36]:
input_size = X_lemmatization_tensor.shape[1]  # Размер входа
hidden_size = 64  # Размер скрытого слоя
output_size = len(np.unique(y_lemm))  # Количество классов

model_lemm_gru = GRUModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lemm_gru.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for reviews, targets in dataloader_lemmatization:
        optimizer.zero_grad()
        outputs = model_lemm_gru(reviews)
        loss_gru_lem = criterion(outputs, targets)
        loss_gru_lem.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_gru_lem.item():.4f}')

Epoch [1/10], Loss: 0.0977
Epoch [2/10], Loss: 0.1835
Epoch [3/10], Loss: 0.0168
Epoch [4/10], Loss: 0.0084
Epoch [5/10], Loss: 0.0056
Epoch [6/10], Loss: 0.0005
Epoch [7/10], Loss: 0.0019
Epoch [8/10], Loss: 0.0004
Epoch [9/10], Loss: 0.0004
Epoch [10/10], Loss: 0.0009


#### Вывод

In [43]:
accuracy_stemm_rnn, f1_stemm_rnn = evaluate_model(model_stemming_rnn, dataloader_stemming)
accuracy_lemm_rnn, f1_lemm_rnn = evaluate_model(model_lemm_rnn, dataloader_lemmatization)
accuracy_stemm_lstm, f1_stemm_lstm = evaluate_model(model_stemming_rnn, dataloader_stemming)
accuracy_lemm_lstm, f1_lemm_lstm = evaluate_model(model_lemm_rnn, dataloader_lemmatization)
accuracy_stemm_gru, f1_stemm_gru = evaluate_model(model_stemming_rnn, dataloader_stemming)
accuracy_lemm_gru, f1_lemm_gru = evaluate_model(model_lemm_rnn, dataloader_lemmatization)

In [44]:
metrics = pd.DataFrame({
    "RNN (stemming)": [accuracy_stemm_rnn * 100, f1_stemm_rnn * 100, loss_rnn_stem.item()],
    "RNN (lemmatization)": [accuracy_lemm_rnn * 100, f1_lemm_rnn * 100, loss_rnn_lem.item()],
    "LSTM (stemming)": [accuracy_stemm_lstm * 100, f1_stemm_lstm * 100, loss_lstm_stem.item()],
    "LSTM (lemmatization)": [accuracy_lemm_lstm * 100, f1_lemm_lstm * 100, loss_lstm_lem.item()],
    "GRU (stemming)": [accuracy_stemm_gru * 100, f1_stemm_gru * 100, loss_gru_stem.item()],
    "GRU (lemmatization)": [accuracy_lemm_gru * 100, f1_lemm_gru * 100, loss_gru_lem.item()]},
    index = ["Accuracy", "F1 score", "Loss"])
metrics

,RNN (stemming),RNN (lemmatization),LSTM (stemming),LSTM (lemmatization),GRU (stemming),GRU (lemmatization)
Accuracy,99.959623,99.959623,99.959623,99.959623,99.959623,99.959623
F1 score,99.959562,99.959574,99.959562,99.959574,99.959562,99.959574
Loss,0.000989,0.001852,0.002097,0.000821,0.000073,0.000934


### Выводы

* Стемминг и лемматизация показала примерно одинаковые результаты.
* Модели рекурентной сети показали результаты лучше, чем методы машинного обучения
* У моделей машинного обучения показатели точности варьировались от 88 до 93